In [1]:
import tqdm

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

import os
import random

import argparse
import json
import pandas as pd
import tqdm
import argparse
import warnings
import joblib

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from src.data import dl_data_load, dl_data_split, dl_data_loader

warnings.filterwarnings(action='ignore')

In [2]:
SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

In [3]:
# Metrics
def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))


class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()
        self.eps = 1e-6

    def forward(self, x, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y)+self.eps)
        return loss

In [4]:
import numpy as np
import torch
import torch.nn as nn


# FM모델 등에서 활용되는 선형 결합 부분을 정의합니다.
class FeaturesLinear(nn.Module):
    def __init__(self, field_dims: np.ndarray, output_dim: int=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)


    def forward(self, x: torch.Tensor):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


# factorization을 통해 얻은 feature를 embedding 합니다.
class FeaturesEmbedding(nn.Module):
    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)


    def forward(self, x: torch.Tensor):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)



# NCF 모델은 MLP와 GMF를 합하여 최종 결과를 도출합니다.
# MLP을 구현합니다.
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)


    def forward(self, x):
        return self.mlp(x)


# Wide: memorization을 담당하는 generalized linear model
# Deep: generalization을 담당하는 feed-forward neural network
# wide and deep model은 위의 wide 와 deep 을 결합하는 모델입니다.
# 데이터를 embedding 하여 MLP 으로 학습시킨 Deep 모델과 parameter에 bias를 더한 linear 모델을 합하여 최종결과를 도출합니다.
class _WideAndDeepModel(nn.Module):
    def __init__(self, args, data):
        super().__init__()
        self.field_dims = data['field_dims']
        self.linear = FeaturesLinear(self.field_dims)
        self.embedding = FeaturesEmbedding(self.field_dims, args.embed_dim)
        self.embed_output_dim = len(self.field_dims) * args.embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, args.mlp_dims, args.dropout)


    def forward(self, x: torch.Tensor):
        embed_x = self.embedding(x)
        x = self.linear(x) + self.mlp(embed_x.view(-1, self.embed_output_dim))
        return x.squeeze(1)


In [5]:
class WideAndDeepModel:

    def __init__(self, args, data):
        super().__init__()

        self.criterion = RMSELoss()

        self.train_dataloader = data['train_dataloader']
        self.valid_dataloader = data['valid_dataloader']
        self.field_dims = data['field_dims']

        self.embed_dim = args.embed_dim
        self.epochs = args.epochs
        self.learning_rate = args.lr
        self.weight_decay = args.weight_decay
        self.log_interval = 100

        self.device = args.device

        self.mlp_dims = args.mlp_dims
        self.dropout = args.dropout

        self.model = _WideAndDeepModel(args,data).to(self.device)
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.learning_rate, amsgrad=True, weight_decay=self.weight_decay)


    def train(self):
      # model: type, optimizer: torch.optim, train_dataloader: DataLoader, criterion: torch.nn, device: str, log_interval: int=100
        for epoch in range(self.epochs):
            self.model.train()
            total_loss = 0
            tk0 = tqdm.tqdm(self.train_dataloader, smoothing=0, mininterval=1.0)
            for i, (fields, target) in enumerate(tk0):
                fields, target = fields.to(self.device), target.to(self.device)
                y = self.model(fields)
                loss = self.criterion(y, target.float())
                self.model.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
                if (i + 1) % self.log_interval == 0:
                    tk0.set_postfix(loss=total_loss / self.log_interval)
                    total_loss = 0

            rmse_score = self.predict_train()
            print('epoch:', epoch, 'validation: rmse:', rmse_score)


    def predict_train(self):
        self.model.eval()
        targets, predicts = list(), list()
        with torch.no_grad():
            for fields, target in tqdm.tqdm(self.valid_dataloader, smoothing=0, mininterval=1.0):
                fields, target = fields.to(self.device), target.to(self.device)
                y = self.model(fields)
                targets.extend(target.tolist())
                predicts.extend(y.tolist())
        return rmse(targets, predicts)


    def predict(self, dataloader):
        self.model.eval()
        predicts = list()
        with torch.no_grad():
            for fields in tqdm.tqdm(dataloader, smoothing=0, mininterval=1.0):
                fields = fields[0].to(self.device)
                y = self.model(fields)
                predicts.extend(y.tolist())
        return predicts

In [6]:
import argparse
# default args
args = argparse.Namespace(
  seed=42,
  data_path = '/opt/ml/data/',
    batch_size = 1024,
    data_shuffle = True,
    test_size = 0.2,
    epochs = 5,
    device = 'cuda',
    lr = 0.001,
    weight_decay = 0.0001,
    embed_dim = 16,
    mlp_dims = [16,16],
    dropout = 0.2,
    num_layers = 3
)

In [7]:
dl_dataset = dl_data_load(args)

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(dl_dataset['train'].drop(['rating'], axis=1), dl_dataset['train']['rating']):
    folds.append((train_idx, valid_idx))

In [9]:
def objective(trial):
    # trial 객체를 이용해 하이퍼파라미터 탐색 공간 설정
    args.batch_size = trial.suggest_categorical('batch_size', [128, 256, 512, 1024])
    args.lr = trial.suggest_loguniform('lr',0.001,0.01)
    args.weight_decay = trial.suggest_loguniform('weight_decay', 1e-8, 1e-3)
    args.embed_dim = trial.suggest_int('embed_dim', 1, 16) # 16이 default
    mlp_dim_layers = trial.suggest_int('mlp_dim_layers',1,4)
    # args.mlp_dims = [trial.suggest_int('mlp_dims',1,20)] * mlp_dim_layers
    args.dropout = trial.suggest_categorical("dropout",[0.2,0.25,0.3])
    dl_data = dl_data_split(args,dl_dataset)
    dl_data = dl_data_loader(args,dl_data)
    model = WideAndDeepModel(args, dl_data)
    model.train()
    loss = model.predict_train()
    return loss

In [10]:
for fold in range(0,5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = dl_dataset['train'].drop(['rating'],axis = 1).iloc[train_idx]
    X_valid = dl_dataset['train'].drop(['rating'],axis = 1).iloc[valid_idx]
    y_train = dl_dataset['train']['rating'].iloc[train_idx]
    y_valid = dl_dataset['train']['rating'].iloc[valid_idx]

    sampler = optuna.samplers.TPESampler(SEED)
    study = optuna.create_study(
        study_name = 'WDN_parameter_opt',
        direction = 'minimize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=5)    
    args.__dict__.update(study.best_params)
    
    dl_dataset['X_train'], dl_dataset['X_valid'], dl_dataset['y_train'], dl_dataset['y_valid'] = X_train, X_valid, y_train, y_valid
    dl_data = dl_data_loader(args,dl_dataset)
    
    model = WideAndDeepModel(args, dl_data)
    model.train()
    
    pred = model.predict(dl_data['test_dataloader'])
    dl_dataset[f'pred_{fold}'] = pred
    print(f'================================================================================\n\n')

[I 2023-04-18 12:19:52,274] A new study created in memory with name: WDN_parameter_opt


====================================1============================================


  0%|          | 0/480 [00:00<?, ?it/s]

epoch: 0 validation: rmse: 2.4232153121331304


  0%|          | 0/480 [00:00<?, ?it/s]

...

  0%|          | 0/959 [00:00<?, ?it/s]

epoch: 3 validation: rmse: 2.240342108672214


  0%|          | 0/300 [00:00<?, ?it/s]

epoch: 4 validation: rmse: 2.2659742684377897


100%|██████████| 300/300 [00:00<00:00, 416.03it/s]